In [363]:
import docx2txt
import re
from collections import Counter
# FILE_PATH=input("Enter template path(File should be of .docx format) ->   ")
FILE_PATH='doc1.docx'
try:
    file_data = docx2txt.process(FILE_PATH)
#     file_data=data
except:
    print("Oh No ! ..Error file name is missing in the path ("+FILE_PATH+") mentioned or error with file format")
    
def error_index_conditions(error_index,test):
    forward_index=20
    ERROR_MESSAGE='Oops ! Bracket seems to have an issue between these characters -> '
    if error_index+forward_index>len(test) and error_index-forward_index>0:
        return ERROR_MESSAGE+test[error_index-forward_index:error_index]
    elif error_index-forward_index<0 and error_index+forward_index<len(test):
        return ERROR_MESSAGE+test[error_index:error_index+forward_index]
    else:
        return ERROR_MESSAGE+test[error_index-forward_index:error_index+forward_index]
def diff_list_keys(l1,l2):
    for i in range(0,len(l1)):
        for j in range(0,len(l2)):
            if l1[i]==l2[j]:
                l1[i]=None
                l2[j]=None
    
    return list(filter(None,l2))
def check_balanced_brackets(test):
    print(10*"*"+"Checking Bracket Count!!"+10*"*")
    stack=[]
    open_list = ["[","("]
    close_list = ["]",")"]
    
    pos=0
    for i in range(0,len(test)):
        
        if test[i] in open_list:
            stack.append([test[i],i])
        elif test[i] in close_list:
        
            pos=close_list.index(test[i])

            if ((len(stack) > 0) and (open_list[pos] == stack[len(stack)-1][0])):
                stack.pop()
            else:
                
                if len(stack)==0:
                    error_index=i
                else:
                    error_index=stack[0][1]
                return error_index_conditions(error_index,test)
    
    if len(stack) == 0:
        return "Brackets count seems to be ok!!"
    else:
        error_index=stack[0][1]
        return error_index_conditions(error_index,test)
    
#Ist rule to check if brackets count is same or if they are balanced or not
data_remove_new_line=re.sub('\n'," ",file_data)
try:
    print(check_balanced_brackets(data_remove_new_line))
except:
    print("ERROR! checking count of brackets ! Please look in the template for BRACKETS")

print(10*"*"+"Checking for logic part!!"+10*"*")
# 2nd RULE : Check logic part of template

data_new_line_split=list(filter(None,file_data.split("\n")))

#Assume 1: There is if and populate in all the data lines
count_expression=0
for data_item in data_new_line_split:
    logic_parts=re.findall('\[\((if)(.*?)(populate\))',data_item,re.IGNORECASE)
    
    for logic_part in logic_parts:
        
        count_expression+=1
# checking logic part 0
        if logic_part[0][0].islower():
            print("Oops ! 'i' in if  is not UpperCase for Logic -> "+"".join(logic_part))
            
# checking logic part 1
        bracket_keys=re.search('\[[A-Z]{1}[a-z]+[A-Z]{1}[a-z]+\]|\[F\d+\]',logic_part[1])

        if bracket_keys:
            print("Logic part has keys with brackets -> "+logic_part[1])
        if logic_part[1][0]!=' ':
            print("Oops! No space between If and expression ->"+"".join(logic_part))
        if re.search('\s{2,}',logic_part[1]):
            print("Oops! More than 2  spaces between If and expression ->"+"".join(logic_part))
        if re.search('logic|of|AND|OR|IS NOT|is not|&|:|\[|\]',logic_part[1]):
            print("Oops! Issue with this part of expression(It contains these Characters - logic|of|AND|OR|IS NOT|is not|&|:) ->"+"".join(logic_part))
        
        if logic_part[1][-1]!=' ' or logic_part[1][-2]!=',':
            print("Oops! No space or comma with this part of expression ->"+"".join(logic_part))
        if re.search(",",logic_part[1][1:-2]):
            print("Oops! comma found with this part of expression ->"+"".join(logic_part))
            
logic_part_else=re.findall('\(else.*?populate\)',data_remove_new_line,re.IGNORECASE)
if len(logic_part_else)!=0:
    print("else part in logic -> ",logic_part_else)
    

    

#Assume 2: There is no if in data lines

count_populate=data_remove_new_line.count("populate)")
count_If=data_remove_new_line.count("(If")
if count_If!=count_populate:
    missing_If_or_populate_count=abs(count_populate-count_If)
    print(str(missing_If_or_populate_count)+" 'If' or 'populate' seems to be missing in some of populate expression")
elif count_If!=count_expression:
    print("There is a issue with the expressions in template Seems like 'If' doesn't start with [(")


#Assume 3: There is word logic in data lines

for data_item in data_new_line_split:
    logic_parts=re.findall('\((logic)(.*?populate\))',data_item,re.IGNORECASE)
    if len(logic_parts)!=0:
        print("Word 'logic' found in this part of expression ->","".join(logic_parts[0]))
        
#Assume 4: There is no if and populate

no_if_and_populate=re.findall('\[\(F\d+[=≠:><].*?\)',data_remove_new_line,re.IGNORECASE)

if len(no_if_and_populate)!=0:
    print("No if and populate found for these logics ->",no_if_and_populate)
    
# 3rd RULE : Right syntax for keys

print(10*"*"+"Checking for Right syntax for keys !!"+10*"*")
search_data_for_keys=re.sub('if.*?populate|logic.*?populate|\[.*?return.*?\]',"",data_remove_new_line,flags=re.IGNORECASE)

keys_exp_without_brack=re.findall('[A-Z]{1}[a-z]+[A-Z]{1}[a-z]+|F\d+',search_data_for_keys)

keys_exp_without_brack=["[" + s +"]" for s in keys_exp_without_brack]
keys_exp_with_brack=re.findall('\[[A-Z]{1}[a-z]+[A-Z]{1}[a-z]+\]|\[F\d+\]',search_data_for_keys)

# keys_error_values=[i for i, j in zip(keys_exp_without_brack, keys_exp_with_brack)  if "["+i+"]" != j ] 

keys_error_values=diff_list_keys(keys_exp_with_brack,keys_exp_without_brack)


if len(keys_error_values)!=0:
    print("Issue with keys not in brackets ->",keys_error_values)

keys=re.findall('\[(.*?F\d+)\]', data_remove_new_line,re.IGNORECASE)
invalid_keys_for_space=[]
for key in keys:
    if not re.search('populate|logic|if|\)|\(',key,re.IGNORECASE):
        space_count=key.count(' ')
        if space_count>0:
            invalid_keys_for_space.append(key)
invalid_keys_with_underscore=re.findall('\[f\d+_f\d+\]', data_remove_new_line,re.IGNORECASE)
invalid_keys=invalid_keys_with_underscore+invalid_keys_for_space
if len(invalid_keys)>0:
    print("These Keys are invalid -> ",invalid_keys)

all_f_keys=re.findall('\[(f\d+)\]',search_data_for_keys,re.IGNORECASE)
for key in all_f_keys:
    if key[0].islower():
        print(key+" Has a lower case 'F'")
        
        
# 4th RULE : Address syntax
print(10*"*"+"Checking for Address syntax!!"+10*"*")
for data_item in data_new_line_split:
    address_part=re.findall('\[[A-Z]+[a-z]+[A-Z]+\w+\]',data_item)
    if len(address_part)==4 and not re.search('\[[A-Z]+[a-z]+[A-Z]+\w+\], \[[A-Z]+[a-z]+[A-Z]+\w+\], \[[A-Z]+[a-z]+[A-Z]+\w+\] \[[A-Z]+[a-z]+[A-Z]+\w+\]',data_item.strip()):
        print("address format seems to be incorrect for string -> "+data_item)
        
    
# 5th RULE : dash check for keys
print(10*"*"+"Checking for dash for keys!!"+10*"*")
for data_item in data_new_line_split:

    dash_check=re.search('\[F\d+\]-\[F\d+\]',data_item)
    
    if dash_check:
        string_r=re.sub('\[|\]','',dash_check.group())
        print("replace dash with ",'['+string_r.split('-')[0]+']'+'[(If '+string_r.split('-')[1]+'≠BLANK, populate)-]['+string_r.split('-')[1]+']')
        
# 6th RULE : Footer issue 
print(10*"*"+"Checking for Footer issue!!"+10*"*")
footer_with_prop=re.findall('proprietary',data_remove_new_line,re.IGNORECASE)
if len(footer_with_prop)!=0:
    print("Remove Proprietary from footer !")
    


**********Checking Bracket Count!!**********
Brackets count seems to be ok!!
**********Checking for logic part!!**********
**********Checking for Right syntax for keys !!**********
**********Checking for Address syntax!!**********
**********Checking for dash for keys!!**********
**********Checking for Footer issue!!**********


In [270]:
diff_list_keys()

NoneType

In [293]:
from collections import Counter

c1 = Counter(l1)
c2 = Counter(l2)

diff = c1-c2
print(list(diff.elements()))

[]


In [303]:
l1=['[ClientName]', '[PlanName]']
l2=['[PlanName]', '[ClientName]', '[PlanName]']


['[PlanName]']

In [295]:
l1


[None, None]